<a href="https://colab.research.google.com/github/ishi720/colab_scraping/blob/master/scraping_sample_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Twitterの検索結果から画像を保存します

GoogleDriveの"scraping_image"を作成して処理します。

すでに"scraping_image"がある場合は、中身が一度削除されてしまうのでご注意ください。




In [0]:
# 環境整備
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

In [0]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# 保存フォルダを作る
!cd drive/'My Drive'/ && mkdir scraping_image

# フォルダの画像を空にする
!cd drive/'My Drive'/scraping_image/ && rm ./* && ls -la

In [0]:
# クロールするURLを作成する
import urllib.parse
keyword = "焼肉　肉"
url = "https://twitter.com/search?f=images&q="+ urllib.parse.quote(keyword) +"&src=typd"
print(url)

https://twitter.com/search?f=images&q=%E7%84%BC%E8%82%89%E3%80%80%E8%82%89&src=typd


In [0]:
# 画像の保存処理

import io
import urllib.request
from PIL import Image

from urllib.parse import urlparse

# 画像を保存する
# url: 保存する画像のURL
# filename: 保存したときのファイル名
def imageSave(url, filename):
  
  #画像を読み込み
  f = io.BytesIO(urllib.request.urlopen(url).read())
  img = Image.open(f)

  # 画像の保存
  img = img.convert("RGB")
  drive_root_dir='/content/drive/My Drive/scraping_image/'
  img.save( drive_root_dir + filename + ".jpg", 'JPEG')

In [0]:
# クロール
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

options.add_argument('--lang=ja')
options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36')

driver = webdriver.Chrome('chromedriver',options=options)

time.sleep(5)

driver.set_script_timeout(30)
driver.get(url)
html = driver.page_source.encode('utf-8')

In [0]:
# スクレイピング
soup = BeautifulSoup(html, "html.parser")

# imgタグ取得
images = soup.find_all('img')

loop = 0
for img in images:
  print( str(loop) + "/" + str(len(images)) )
  loop = loop + 1
  if 'src' in img.attrs:
    imageSave(img.attrs['src'], str(loop) )
  elif 'data-src' in img.attrs:
    imageSave(img.attrs['data-src'], str(loop) )